# Конвертация типов

In [1]:
import cv2
import convertapi
from PIL import Image

In [2]:
import struct
import zlib

from typing import Callable

In [3]:
IMG_PATH = './materials/imgs/blackbuck.bmp'

RESULTS_PATH = './tmp_results/'

## Простое задание
1. Конвертировать файл bmp в png любым способом, а также написать плюсы и минусы используемого метода/библиотеки.
1. Поменять чётные строки файла bmp на синий цвет, считав файл как массив данных побитно

*Некоторые сопсобы конвертирования изображения:*


| Method                    | Workflow            | +            | -            |
|---------------------------|----------------|--------------|--------------|
| OpenCV2                   | Преобразует в массив NumPy, представляющий изображение, которое можно обрабатывать далее  | - Самая популярна и доступна<br>- Быстрая и эффективная<br>- Возможность обработки изображения перед сохранением | - Возможно много лишнего фукнционала под эту конкретную задачу |
| Pillow                    | Pillow считывает в растровом виде и хранит в личном типе данных со своими метаданными     | - Также довольно популярна | - Может работать медленнее |
| ConvertAPI                |                                                                                           | - Обработка изображения на стороне сервиса | - Необходима регистрация<br>- Зависимость от работоспособности сервера и интернета<br>- Расширенный функционал только платно |
| Личный побитный конвертер |    Данные представленны в виде списка bytearray                                           | - Полное понимание процесса "под капотоп"<br>- Респект и уважение :) | - Риск напортачить |

Далее представлен код каждого из методов + замена черного цвета на синий.

Результаты расположены в директории tmp_results

In [4]:
# OpenCV2

img = cv2.imread(IMG_PATH)

cv2.imwrite(RESULTS_PATH + 'cv2_blackbuck.png', img)

True

In [5]:
# Pillow

img = Image.open(IMG_PATH)

img.save(RESULTS_PATH + 'pil_blackbuck.png')

In [ ]:
# ConvertAPI

convertapi.api_credentials = 'your-api-secret-or-token' # Pass your token after registration

result = convertapi.convert('png', {'File': IMG_PATH})
result.file.save(RESULTS_PATH + 'api_blackbuck.png')

**Replace black to blue in every second picture's row**

In [6]:
def read_bmp(img_path: str):
    '''Read file bit by bit and return rows'''

    with open(img_path, 'rb') as f:
        img_head = f.read(54)

        # Extract info about img size in little-endian format: 
        # info about size is started from 18 + 2 int (4 bytes) = 18+2*4 = before 2
        width, height = struct.unpack('<II', img_head[18:26])

        # Every row should be a multiple of 4 bytes. So, we add 3 bytes
        # and use bitwise operations to round down to the nearest multiple of 4
        row_size = (width * 3 + 3) & (~3)
        print(f'Width: {width}, Height: {height}, Row_size: {row_size}')

        pixels_data = []

        for _ in range(height):
            row = f.read(row_size)
            pixels_data.append(row)


    img_metadata = {
        'header': img_head,
        'width': width,
        'height': height,
        'row_size': row_size,
        'pixels_data': pixels_data
    }

    return img_metadata

In [7]:
def change_color(
    pixels_data: list, replace_colors: tuple, row_step: int=2
):
    ''' 
        replace_colors_dict: tuple(tuple)
                (old_color_value_in_bgr, new_color_value_in_bgr)
        
        row_step: int
                1 - replace in every row
                2 - replace in every second row and etc.
    '''
    changes_pixels_data = pixels_data.copy()

    for i in range(0, len(changes_pixels_data), row_step):
        row = bytearray(changes_pixels_data[i])

        for j in range(0, len(row), 3):
            color_bgr = (row[j], row[j+1], row[j+2])
            if color_bgr == replace_colors[0]:
                row[j], row[j+1], row[j+2] = replace_colors[1]

        changes_pixels_data[i] = bytes(row)

    return changes_pixels_data

In [8]:
img_metadata = read_bmp(IMG_PATH)

# BGR: black - (0,0,0), blue - (255, 0, 0)
pixels_data = change_color(
    img_metadata['pixels_data'], 
    replace_colors=((0, 0, 0), (255, 0, 0),),
    # row_step=10
)

with open(img_result := RESULTS_PATH + 'custom_change_color_blackbuck.bmp', 'wb') as f:
    f.write(img_metadata['header'])
    for row in pixels_data:
        f.write(row)

print(f'Updated black color in img {IMG_PATH} to {img_result}')

Width: 512, Height: 512, Row_size: 1536
Updated black color in img ./materials/imgs/blackbuck.bmp to ./tmp_results/custom_change_color_blackbuck.bmp


## Альтернативное задание
1. Написать свой конвертер из png в bmp файл, используя только считывание файлов побитно через массивы

In [9]:
class ConverterBmp2Png:

    @classmethod
    def __get_data_in_big_endian(self, data: str) -> bytes:
        return struct.pack('>I', data)

    @classmethod
    def __get_chunk(self, chunk_data: bytearray, chunk_type: str) -> bytearray:

        chunk = bytearray()
        
        # data_size
        chunk.extend(
            ConverterBmp2Png.__get_data_in_big_endian(len(chunk_data))
        )
        # chunk_type
        chunk.extend(
            chunk_type_byte := chunk_type.encode('utf-8')
        )
        # chunk_data
        chunk.extend(chunk_data)
        # CRC
        chunk.extend(
            ConverterBmp2Png.__get_data_in_big_endian(
                zlib.crc32(chunk_type_byte + chunk_data)
            )
        )

        return chunk


    @classmethod
    def __create_IHDR_chunk(self, width, height) -> bytearray:
        indr_chunk = bytearray()

        # width
        indr_chunk.extend(
            ConverterBmp2Png.__get_data_in_big_endian(width)
        )
        # height
        indr_chunk.extend(
            ConverterBmp2Png.__get_data_in_big_endian(height)
        )
        # bit depth
        indr_chunk.append(8)
        # RGB color type
        indr_chunk.append(2)
        # compression_method 
        indr_chunk.append(0)
        # filter_method 
        indr_chunk.append(0)
        # interlace_method
        indr_chunk.append(0)
        
        return indr_chunk
    

    @classmethod
    def __create_IDAT_chunk(self, bmp_pixel_data) -> bytearray:
        idat_chunk = bytearray()

        # PNG in botton-top
        for row in reversed(bmp_pixel_data):
            # None (0): No filtering is applied. The raw pixel data is sent directly to compression.
            idat_chunk.append(0)
            for i in range(0, len(row), 3):
                # BGR to RGB
                idat_chunk.extend([row[i+2], row[i+1], row[i]])
        
        idat_chunk = zlib.compress(idat_chunk)
        
        return idat_chunk
    
    
    @classmethod
    def __get_png_data(self, img_bmp_metadata: dict) -> bytearray:
        png_header = b'\x89' + b'PNG' + b'\r\n' + b'\x1a' + b'\n'

        ihdr_chunk = ConverterBmp2Png.__create_IHDR_chunk(
            img_bmp_metadata['width'], img_bmp_metadata['height']
        )

        idat_chunk = ConverterBmp2Png.__create_IDAT_chunk(
            img_bmp_metadata['pixels_data']
        )

        all_chunks = \
            png_header + \
            ConverterBmp2Png.__get_chunk(ihdr_chunk, 'IHDR') + \
            ConverterBmp2Png.__get_chunk(idat_chunk, 'IDAT') + \
            ConverterBmp2Png.__get_chunk(b'', 'IEND')

        return all_chunks


    @staticmethod
    def lets_convert(
        bmp_img_path: str=IMG_PATH,
        png_img_path: str=RESULTS_PATH+'custom_converted_blackbuck.png',
        fnc_get_bmp_metadata: Callable[[str], dict]=read_bmp,
    ) -> None:
        img_bmp_metadata = fnc_get_bmp_metadata(bmp_img_path)

        png_data = ConverterBmp2Png.__get_png_data(img_bmp_metadata)

        with open(png_img_path, 'wb') as f:
            f.write(png_data)

        print(f'Image {bmp_img_path} success saved to {png_img_path}')

In [10]:
ConverterBmp2Png.lets_convert()

Width: 512, Height: 512, Row_size: 1536
Image ./materials/imgs/blackbuck.bmp success saved to ./tmp_results/custom_converted_blackbuck.png
